# Internet and Websites Search using Bing API - Bing Chat Clone

이 노트북에서는, Langchain과 Azure Bing Search API 서비스를 모두 활용하여 웹 검색 기능으로 GPT Smart Search Engine을 부스팅할 수 있는 방법에 대해 알아보겠습니다.

앞서, 다른 노트북에서 설명했듯이 harnessing agent 와 tools는 효과적인 접근법입니다. 우리는 GPT-4 및 그 후속제품과 같은 OpenAI의 대규모 언어모델(LLM)을 활용하여 추론 및 연구를 대신하여 수행하는 것을 목표로 하고 있습니다.

스마트 검색 엔진에 인터넷 접속이 필요한 경우는 여러 가지가 있습니다. 예를 들어, **웹에서 이용 가능한 정보를 제공하여 답변을 풍부하게 하거나**, **사용자에게 최신 및 최근 정보를 제공하거나**, **특정 공개 웹사이트에서 정보를 찾는 등**의 경우에도 검색 결과를 기반으로 엔진의 응답을 요구할 수 있습니다.

이 노트를 진행하면 **Bing Search API를 사용하여 Web Search Agent를 생성하는 방법**과 이러한 도구가 챗봇을 강화하는 방법 등 Bing Search API 기본 사항에 대해 확실하게 이해할 수 있을 것입니다. 또한 **Callback Handler, 사용법 및 봇 애플리케이션에서의 중요성**에 대해서도 알게 될 것입니다.

In [4]:
import requests
import os
from typing import Dict, List
from pydantic import BaseModel, Extra, root_validator

from langchain.chat_models import AzureChatOpenAI
from langchain.agents import AgentExecutor
from langchain.callbacks.manager import CallbackManager
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool
from langchain.utilities import BingSearchAPIWrapper

from common.callbacks import StdOutCallbackHandler
from common.prompts import BING_PROMPT_PREFIX

from IPython.display import Markdown, HTML, display  

from dotenv import load_dotenv
load_dotenv("credentials.env")

def printmd(string):
    display(Markdown(string.replace("$","USD ")))

# 이 코드를 사용하려면 GPT-4 모델이 필요합니다. GPT-35-터보는 Prompt에 따라 오류가 나올 수 있습니다. 
MODEL_DEPLOYMENT_NAME = "gpt-4" 

In [5]:
# Langchain을 사용하여 Azure OpenAI에 연결하는 데 필요한 ENV 변수를 설정
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"]
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"]
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]
os.environ["OPENAI_API_TYPE"] = "azure"

## Introduction to Callback Handlers


다음 설명은 Callback에 대한 Langchain 문서에서 직접 가져온 자료입니다.([링크](https://python.langchain.com/docs/modules/callbacks/)):

**Callbacks**:<br>
LangChain은 LLM 응용 프로그램의 다양한 단계에 연결할 수 있는 콜백 시스템을 제공합니다. 이는 로그, 모니터링, 스트리밍 및 기타 작업에 유용합니다. API를 통해 사용 가능한 Callback 인수를 사용하여 이러한 이벤트를 구독할 수 있습니다. 이 인수는 핸들러 개체의 목록입니다.

**Callback handlers**:<br>
CallbackHandler는 CallbackHandler 인터페이스를 구현하는 개체로, 구독할 수 있는 각 이벤트에 대한 메서드가 있습니다. CallbackManager는 이벤트가 트리거되면 각 핸들러에서 해당 메서드를 호출합니다.

--------------------
콜백에 대한 핸들러를 통합하여 응답이 스트리밍될 때, 이를 관찰하고 에이전트의 추론 과정에 대한 통찰력을 얻을 수 있습니다. 이는 봇의 응답을 사용자에게 스트리밍하고 응답을 기다리는 동안 진행 중인 프로세스에 대해 계속 정보를 제공하는 것을 목포료 할 때 매우 유용합니다.

저희의 커스텀 핸들러는 `common/callbacks.py` 에서 확인하실 수 있습니다.

In [6]:
cb_handler = StdOutCallbackHandler()
cb_manager = CallbackManager(handlers=[cb_handler])

# 콜백 핸들러와 함께 LLM 개체를 선언합니다.
llm = AzureChatOpenAI(deployment_name=MODEL_DEPLOYMENT_NAME, temperature=0, max_tokens=1000)

# 혹은 응답이 스트리밍 되는 것을 보고 싶다면 아래 코드에서 코멘트를 해제합니다. 
# llm = AzureChatOpenAI(deployment_name=MODEL_DEPLOYMENT_NAME, temperature=0, max_tokens=1000, streaming=True, callback_manager=cb_manager)

## Creating a custom tool - Bing Search API tool

Langhain에는 이미 BingSearchAPIWrapper ([HERE](https://github.com/hwchase17/langchain/blob/master/langchain/utilities/bing_search.py))라는 미리 만들어진 툴이 있지만, 실행 기능 대신 결과 기능을 사용함으로써 좀 더 나은 방법으로 텍스트 결과뿐만 아니라 각 스니펫의 제목과 링크(소스)도 얻을 수 있습니다.

In [7]:
class MyBingSearch(BaseTool):
    """Tool for a Bing Search Wrapper"""
    
    name = "@bing"
    description = "useful when the questions includes the term: @bing.\n"

    k: int = 5
    
    def _run(self, query: str) -> str:
        bing = BingSearchAPIWrapper(k=self.k)
        return bing.results(query,num_results=self.k)
            
    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("This Tool does not support async")

이제 맞춤형 도구와 맞춤형 프롬프트 `BING_PROMPT_PREFIX`를 사용하는 REACT Agent를 생성합니다.

In [8]:
tools = [MyBingSearch(k=5)]
agent_executor = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         agent_kwargs={'prefix':BING_PROMPT_PREFIX}, callback_manager=cb_manager, )

아래 질문들 중 일부를 시도해보시거나, 좋아할만 한 다른 질문들을 시도해보세요.

In [9]:
QUESTION = "현재 모든 산업에서 LLM모델의 전망은 어떻게 돼?"
# QUESTION = "How much is 50 USD in Euros and is it enough for an average hotel in Madrid?"
# QUESTION = "My son needs to build a pinewood car for a pinewood derbi, how do I build such a car?"
# QUESTION = "Who won the 2023 superbowl and who was the MVP?"
# QUESTION = "can I travel to Hawaii, Maui from Dallas, TX for 7 days with $7000 on the month of September, what are the best days to travel?"


# This complex question below needs gpt-4-32k (0613 version) in order to ensure a good answer. 
# ---------------
# QUESTION = """
# compare the number of job opennings (provide the exact number), the average salary within 15 miles of Dallas, TX, for these ocupations:

# - ADN Registerd Nurse 
# - Occupational therapist assistant
# - Dental Hygienist
# - Graphic Designer
# - Real Estate Agent


# Create a table with your findings. Place the sources on each cell.
# """

In [10]:
#As LLMs responses are never the same, we do a for loop in case the answer cannot be parsed according to our prompt instructions
for i in range(2):
    try:
        response = agent_executor.run(QUESTION) 
        break
    except Exception as e:
        response = str(e)
        continue

The user is asking about the current outlook of the LLM model in all industries. Since the question is in Korean, I will respond in Korean as well. I need to understand what the LLM model is and how it is applied across various industries. I will use the @bing tool to find this information.
Action: @bing
Action Input: LLM 모델이란 무엇인가
The LLM (Large Language Model) is an advanced AI algorithm trained with vast data for natural language processing (NLP) tasks. It learns from a large amount of text data to understand and generate human language. The marketability and value of LLMs are receiving more attention with the recent focus on generative AI capabilities. Now, I need to find out about the current outlook of the LLM model in all industries.
Action: @bing
Action Input: 모든 산업에서 LLM 모델의 전망


In [11]:
printmd(response)

LLM 모델은 그것의 예상되는 중요한 영향력 때문에 많은 기업과 조직들의 주요 관심사로 부상하고 있습니다. 미래의 경쟁력을 고려하는 기업들은 차원이 다른 내부 직원용 지식 서비스 (Knowledge Graph) 체계를 구축하거나, 또는 엄청나게 똑똑한 마케팅 혹은 마치 인간처럼 대화하는 고객 커뮤니케이션 서비스를 원할 수 있습니다. 이것을 성공적으로 구축했을 때, 새로운 차원의 경쟁력이 될 수 있습니다. LLM이 계속 발전함에 따라, 고객 서비스와 콘텐츠 생성에서 교육과 연구에 이르기까지 산업 전반에 걸쳐 다양한 애플리케이션을 향상시키고 자동화할 수 있는 큰 잠재력을 가지고 있습니다. 그러나, 기술 발전에 따라 편향된 행동이나 오용과 같은 윤리적 및 사회적 문제도 해결해야 합니다. 또한, LLM 모델은 헬스케어 분야에서 다양한 활용을 예상하고 있습니다. 강력한 LLM이 기존의 합성곱 AI 모델을 대체함에 따라, 트랜스포머는 컴퓨터 비전을 새롭게 정의하고 있습니다. "모든 기업은 AI 기업이다"라는 말이 생각보다 빠르게 우리의 현실이 되고 있습니다. 오랜 시간 동안, 아주 운이 좋은 일부만이 강력한 AI 솔루션에 접근할 수 있었습니다.<sup><a href="https://aisummit.co.kr/2023-llm-newsletter/" target="_blank">[1]</a></sup> <sup><a href="https://ko.shaip.com/blog/a-guide-large-language-model-llm/" target="_blank">[2]</a></sup> <sup><a href="https://blogs.nvidia.co.kr/2022/11/23/llms-ai-horizon/" target="_blank">[3]</a></sup> <sup><a href="https://www.uipath.com/ko/blog/ai/llm-ko-kr" target="_blank">[4]</a></sup>. 다른 질문이 있으신가요?

## 특정 웹 사이트에 대한 QnA

스마트 봇이 특정 회사의 공개 웹 사이트에 대한 질문에 대답하기를 원하는 여러가지 활용 사례가 있으며, 우리가 취할 수 있는 두 가지 접근 방법은 다음과 같습니다 :

1. 정기적으로 실행되는 크롤러 스크립트를 만들어 웹사이트의 모든 페이지를 찾아내고 문서를 Azure Cognitive Search로 푸시합니다.
2. Bing이 이미 공개 웹사이트를 색인화하였을 가능성이 높기 때문에, 사이트를 직접 색인화하고 Bing 크롤러가 이미 수행한 작업을 복제하는 대신 해당 사이트를 대상으로 하는 Bing 검색을 활용할 수 있습니다.

다음은 특정 사이트와 관련된 몇 가지 예시 질문입니다. 다음을 살펴 보세요 :

In [15]:
# QUESTION = "microsoft.com 사이트에서 양자 컴퓨팅에 대한 최신 뉴스를 검색해줘"
QUESTION = "samsung.com 에서 가장 최근에 하는 이벤트를 검색해줘"

In [16]:
#As LLMs responses are never the same, we do a for loop in case the answer cannot be parsed according to our prompt instructions
for i in range(3):
    try:
        response = agent_executor.run(QUESTION) 
        break
    except Exception as e:
        response = str(e)
        continue

사용자는 samsung.com에서 가장 최근의 이벤트에 대해 묻고 있습니다. 이 정보를 얻기 위해 웹 검색을 수행해야 합니다. 
Action: @bing
Action Input: site:samsung.com 이벤트


In [17]:
printmd(response)

samsung.com에서 현재 진행 중인 이벤트는 다음과 같습니다:
1. "삼성닷컴 10주년 페스타": 최대 105만원 상당의 구매 혜택을 제공합니다<sup><a href="https://www.samsung.com/sec/event/10th_festa/buy/" target="_blank">[1]</a></sup>.
2. "Galaxy Watch6 Series 론칭 혜택": T마일리지를 받을 수 있는 이벤트입니다<sup><a href="https://www.samsung.com/sec/event/galaxywatch6/" target="_blank">[2]</a></sup>.
3. "Galaxy Z Fold5｜Z Flip5 구매 고객님께 드리는 특별한 혜택" 이벤트<sup><a href="https://www.samsung.com/sec/event/galaxy-z5/promotion/" target="_blank">[3]</a></sup>.
이 외에도 samsung.com에서는 다양한 이벤트를 진행하고 있습니다<sup><a href="https://www.samsung.com/sec/digitalplaza/eventList/" target="_blank">[4]</a></sup>. 이벤트의 세부 사항과 참여 방법은 각 이벤트의 웹페이지를 참조하시기 바랍니다. 다른 질문이 있으신가요?

In [17]:
# Uncomment if you want to take a look at the custom bing search prompt (This is where the magic happens: a great system promp + GPT-4)
# printmd(agent_executor.agent.llm_chain.prompt.template)

# Summary

이 노트북에서는 콜백 핸들러에 대해 알아보고 LLM에서 응답을 스트리밍하는 방법에 대해 알아보았습니다. 또한 특정 검색 및 형식 지정 지침과 함께 영리한 프롬프트를 사용하여 Bing Chat 클론을 만드는 방법에 대해 알아봅니다.

결과는 당사를 대신하여 지능적인 웹 검색 및 연구 수행이 가능한 에이전트이며, 해당 URL 인용 및 링크와 함께 질문에 대한 답변을 제공합니다!